In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve, auc
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import os

output_dir = 'ex1'
os.makedirs(output_dir, exist_ok=True)

class SVHNDataModule:
    def __init__(self):
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.4377, 0.4438, 0.4728), std=(0.1980, 0.2010, 0.1970))
        ])

    def load_data(self, batch_size):
        train_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=self.transform)
        test_dataset = datasets.SVHN(root='./data', split='test', download=True, transform=self.transform)

        train_size = int(0.8 * len(train_dataset))
        val_size = len(train_dataset) - train_size
        train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        return train_loader, val_loader, test_loader


class SmallVGG(nn.Module):
    def __init__(self):
        super(SmallVGG, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(8, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(32 * 4 * 4, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x


def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device='cuda'):
    model.train()
    device = torch.device(device)
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)
        val_loss = evaluate_model(model, val_loader, device, return_loss=True)
        val_losses.append(val_loss)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

    return train_losses, val_losses


def evaluate_model(model, data_loader, device='cuda', return_loss=False):
    model.eval()
    running_loss = 0.0
    all_labels = []
    all_preds = []
    all_probs = []
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            probs = torch.softmax(outputs, dim=1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    avg_loss = running_loss / len(data_loader)
    accuracy = np.mean(np.array(all_labels) == np.array(all_preds))
    num_classes = 10
    all_labels_onehot = np.eye(num_classes)[all_labels]
    all_probs = np.array(all_probs)
    micro_roc_auc = roc_auc_score(all_labels_onehot, all_probs, average='micro')
    macro_roc_auc = roc_auc_score(all_labels_onehot, all_probs, average='macro')
    print(f'Accuracy: {accuracy * 100:.2f}%, Micro ROC AUC: {micro_roc_auc:.4f}, Macro ROC AUC: {macro_roc_auc:.4f}')
    if return_loss:
        return avg_loss
    else:
        return avg_loss, accuracy, micro_roc_auc, macro_roc_auc, all_labels, all_preds, all_probs

def grid_search_hyperparameters(device='cuda'):
    learning_rates = [0.001, 0.0001]
    batch_sizes = [64, 128]
    num_epochs_list = [10, 50, 200]

    results = []

    # 遍历所有的超参数组合
    for lr, batch_size, num_epochs in itertools.product(learning_rates, batch_sizes, num_epochs_list):
        experiment_details = f"lr={lr}, batch_size={batch_size}, epochs={num_epochs}"
        print(f"Training with {experiment_details}")

        # 数据加载
        data_module = SVHNDataModule()
        train_loader, val_loader, test_loader = data_module.load_data(batch_size)

        # 初始化模型、损失函数和优化器
        model = SmallVGG().to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        # 训练模型
        train_losses, val_losses = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=num_epochs, device=device)

        # 评估模型在测试集上的表现
        test_loss, accuracy, micro_roc_auc, macro_roc_auc, all_labels, all_preds, all_probs = evaluate_model(model, test_loader, device)

        # 绘制 Train Loss 和 Validation Loss 并保存
        plt.figure(figsize=(10, 5))
        plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss')
        plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title(f'Train and Validation Loss for {experiment_details}')
        plt.legend()
        loss_plot_path = os.path.join(output_dir, f'loss_{lr}_{batch_size}_{num_epochs}.png')
        plt.savefig(loss_plot_path)
        plt.close()

        # 绘制 ROC 曲线并保存
        fpr, tpr, _ = roc_curve(np.eye(10)[all_labels].ravel(), np.array(all_probs).ravel())
        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {micro_roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'ROC Curve for {experiment_details}')
        plt.legend(loc="lower right")
        roc_plot_path = os.path.join(output_dir, f'roc_{lr}_{batch_size}_{num_epochs}.png')
        plt.savefig(roc_plot_path)
        plt.close()

        # 绘制混淆矩阵并保存
        cm = confusion_matrix(all_labels, all_preds)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title(f'Confusion Matrix for {experiment_details}')
        cm_plot_path = os.path.join(output_dir, f'confusion_matrix_{lr}_{batch_size}_{num_epochs}.png')
        plt.savefig(cm_plot_path)
        plt.close()

        # 计算精确率和召回率
        precision = np.diag(cm) / np.sum(cm, axis=0)
        recall = np.diag(cm) / np.sum(cm, axis=1)

        # 记录实验结果
        results.append({
            'Learning Rate': lr,
            'Batch Size': batch_size,
            'Epochs': num_epochs,
            'Test Loss': test_loss,
            'Accuracy': accuracy,
            'Micro ROC AUC': micro_roc_auc,
            'Macro ROC AUC': macro_roc_auc,
            'Precision': np.nanmean(precision),
            'Recall': np.nanmean(recall)
        })

    # 绘制精确率-召回率柱状图
    metrics_df = pd.DataFrame(results)
    plt.figure(figsize=(12, 6))
    metrics_df[['Precision', 'Recall']].plot(kind='bar', figsize=(14, 7))
    plt.xticks(
        ticks=range(len(metrics_df)),
        labels=metrics_df[['Learning Rate', 'Batch Size', 'Epochs']].apply(
            lambda row: f"lr={row['Learning Rate']}, bs={row['Batch Size']}, ep={row['Epochs']}", axis=1
        ),
        rotation=45,
        ha='right'
    )
    plt.title('Precision and Recall by Hyperparameter Settings')
    plt.ylabel('Score')
    precision_recall_plot_path = os.path.join(output_dir, 'precision_recall_comparison.png')
    plt.savefig(precision_recall_plot_path)
    plt.close()

    # 将结果保存到 ex1 目录中的 CSV 文件
    results_csv_path = os.path.join(output_dir, 'hyperparameter_tuning_results.csv')
    metrics_df.to_csv(results_csv_path, index=False)
    print(f"结果已保存到 {results_csv_path}")
# 主函数
if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    grid_search_hyperparameters(device=device)


Training with lr=0.001, batch_size=64, epochs=10
Accuracy: 84.83%, Micro ROC AUC: 0.9851, Macro ROC AUC: 0.9843
Epoch [1/10], Train Loss: 1.303923, Val Loss: 0.495875
Accuracy: 89.48%, Micro ROC AUC: 0.9919, Macro ROC AUC: 0.9912
Epoch [2/10], Train Loss: 0.417262, Val Loss: 0.349478
Accuracy: 90.85%, Micro ROC AUC: 0.9934, Macro ROC AUC: 0.9929
Epoch [3/10], Train Loss: 0.334015, Val Loss: 0.309632
Accuracy: 90.46%, Micro ROC AUC: 0.9932, Macro ROC AUC: 0.9929
Epoch [4/10], Train Loss: 0.292320, Val Loss: 0.318851
Accuracy: 91.50%, Micro ROC AUC: 0.9939, Macro ROC AUC: 0.9936
Epoch [5/10], Train Loss: 0.260541, Val Loss: 0.295484
Accuracy: 91.01%, Micro ROC AUC: 0.9940, Macro ROC AUC: 0.9937
Epoch [6/10], Train Loss: 0.238997, Val Loss: 0.296160
Accuracy: 91.78%, Micro ROC AUC: 0.9942, Macro ROC AUC: 0.9939
Epoch [7/10], Train Loss: 0.216976, Val Loss: 0.290043
Accuracy: 91.12%, Micro ROC AUC: 0.9938, Macro ROC AUC: 0.9938
Epoch [8/10], Train Loss: 0.197991, Val Loss: 0.322038
Accurac

PermissionError: [Errno 13] Permission denied: 'ex1\\hyperparameter_tuning_results.csv'

<Figure size 1200x600 with 0 Axes>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

output_dir = 'ex2'
os.makedirs(output_dir, exist_ok=True)

# SVHN Data Module
class SVHNDataModule:
    def __init__(self):
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.4377, 0.4438, 0.4728), std=(0.1980, 0.2010, 0.1970))
        ])

    def load_data(self, batch_size):
        train_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=self.transform)
        test_dataset = datasets.SVHN(root='./data', split='test', download=True, transform=self.transform)

        train_size = int(0.8 * len(train_dataset))
        val_size = len(train_dataset) - train_size
        train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        return train_loader, val_loader, test_loader

# Define Small VGG Model
class SmallVGG(nn.Module):
    def __init__(self):
        super(SmallVGG, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(8, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(32 * 4 * 4, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

# Training function with optional early stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50, patience=None, device='cuda', early_stop_start_epoch=30):
    model.train()
    device = torch.device(device)
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)

        # Evaluate on validation set
        val_loss = evaluate_model(model, val_loader, device, return_loss=True)
        val_losses.append(val_loss)
        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

        # Early stopping check (enabled if `patience` is set)
        if patience is not None and epoch >= early_stop_start_epoch:
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                # Save best model
                torch.save(model.state_dict(), 'best_model.pth')
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print("Early stopping triggered")
                    break
        elif patience is None:
            torch.save(model.state_dict(), 'best_model_no_early_stopping.pth')

    return train_losses, val_losses

# Evaluation function
def evaluate_model(model, data_loader, device='cuda', return_loss=False):
    model.eval()
    running_loss = 0.0
    all_labels = []
    all_preds = []
    all_probs = []
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            probs = torch.softmax(outputs, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = running_loss / len(data_loader)
    accuracy = np.mean(np.array(all_labels) == np.array(all_preds))

    if return_loss:
        return avg_loss
    else:
        return avg_loss, accuracy, all_labels, all_preds, all_probs

def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    batch_size = 128
    learning_rate = 0.0005
    num_epochs = 50

    # Load data
    data_module = SVHNDataModule()
    train_loader, val_loader, test_loader = data_module.load_data(batch_size)

    # Model, criterion, and optimizer
    model = SmallVGG().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train model with early stopping
    print("Training with early stopping:")
    patience = 5
    train_losses_es, val_losses_es = train_model(
        model, train_loader, val_loader, criterion, optimizer, num_epochs=num_epochs, patience=patience, device=device
    )

    # Test the model trained with early stopping
    model.load_state_dict(torch.load('best_model.pth'))
    test_loss_es, accuracy_es, all_labels_es, all_preds_es, _ = evaluate_model(model, test_loader, device)

    # Train model without early stopping
    print("\nTraining without early stopping:")
    model = SmallVGG().to(device)  # Reinitialize model
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    train_losses_no_es, val_losses_no_es = train_model(
        model, train_loader, val_loader, criterion, optimizer, num_epochs=num_epochs, patience=None, device=device
    )

    # Test the model trained without early stopping
    model.load_state_dict(torch.load('best_model_no_early_stopping.pth'))
    test_loss_no_es, accuracy_no_es, all_labels_no_es, all_preds_no_es, _ = evaluate_model(model, test_loader, device)

    # Plot training and validation losses for both cases
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses_es, label='Train Loss (Early Stopping)')
    plt.plot(val_losses_es, label='Validation Loss (Early Stopping)')
    plt.plot(train_losses_no_es, label='Train Loss (No Early Stopping)')
    plt.plot(val_losses_no_es, label='Validation Loss (No Early Stopping)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Train and Validation Loss Comparison')
    plt.legend()
    plt.savefig(os.path.join(output_dir, 'train_val_loss_comparison.png'))
    plt.close()

    # Print results for both cases
    print(f"Test Accuracy with Early Stopping: {accuracy_es * 100:.2f}%")
    print(f"Test Accuracy without Early Stopping: {accuracy_no_es * 100:.2f}%")

if __name__ == "__main__":
    main()


Training with early stopping:
Epoch [1/50], Train Loss: 1.480955, Val Loss: 0.761919
Epoch [2/50], Train Loss: 0.604997, Val Loss: 0.519917
Epoch [3/50], Train Loss: 0.449691, Val Loss: 0.455123
Epoch [4/50], Train Loss: 0.376425, Val Loss: 0.396634
Epoch [5/50], Train Loss: 0.328962, Val Loss: 0.357455
Epoch [6/50], Train Loss: 0.294710, Val Loss: 0.343161
Epoch [7/50], Train Loss: 0.268193, Val Loss: 0.331345
Epoch [8/50], Train Loss: 0.245416, Val Loss: 0.325697
Epoch [9/50], Train Loss: 0.225722, Val Loss: 0.319676
Epoch [10/50], Train Loss: 0.206910, Val Loss: 0.315899
Epoch [11/50], Train Loss: 0.188776, Val Loss: 0.336337
Epoch [12/50], Train Loss: 0.172044, Val Loss: 0.346223
Epoch [13/50], Train Loss: 0.158769, Val Loss: 0.321149
Epoch [14/50], Train Loss: 0.143932, Val Loss: 0.339691
Epoch [15/50], Train Loss: 0.125809, Val Loss: 0.352704
Epoch [16/50], Train Loss: 0.114289, Val Loss: 0.359308
Epoch [17/50], Train Loss: 0.099919, Val Loss: 0.401707
Epoch [18/50], Train Loss: 

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.metrics import precision_score, recall_score
# SVHN 数据模块类，接受数据增强参数
class SVHNDataModule:
    def __init__(self, augmentation=None):
        if augmentation is None:
            augmentation = []

        # 基础数据处理
        transform_list = [transforms.ToTensor(),
                          transforms.Normalize(mean=(0.4377, 0.4438, 0.4728), std=(0.1980, 0.2010, 0.1970))]

        # 应用数据增强
        transform_list = augmentation + transform_list
        self.transform = transforms.Compose(transform_list)

    def load_data(self, batch_size):
        train_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=self.transform)
        test_dataset = datasets.SVHN(root='./data', split='test', download=True, transform=self.transform)

        train_size = int(0.8 * len(train_dataset))
        val_size = len(train_dataset) - train_size
        train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        return train_loader, val_loader, test_loader


# 定义小型 VGG 模型
class SmallVGG(nn.Module):
    def __init__(self):
        super(SmallVGG, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(8, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(32 * 4 * 4, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x


# 训练模型的函数
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20, device='cuda'):
    model.train()
    device = torch.device(device)
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)

        # 评估模型在验证集上的损失
        val_loss = evaluate_model(model, val_loader, device, return_loss=True)
        val_losses.append(val_loss)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

    return train_losses, val_losses


def evaluate_model(model, data_loader, device='cuda', return_loss=False):
    model.eval()
    running_loss = 0.0
    all_labels = []
    all_preds = []
    all_probs = []
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            probs = torch.softmax(outputs, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = running_loss / len(data_loader)
    accuracy = np.mean(np.array(all_labels) == np.array(all_preds))

    num_classes = 10
    all_labels_onehot = np.eye(num_classes)[all_labels]
    all_probs = np.array(all_probs)

    micro_roc_auc = roc_auc_score(all_labels_onehot, all_probs, average='micro')
    macro_roc_auc = roc_auc_score(all_labels_onehot, all_probs, average='macro')

    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')

    print(f'Accuracy: {accuracy * 100:.2f}%, Micro ROC AUC: {micro_roc_auc:.4f}, Macro ROC AUC: {macro_roc_auc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}')

    if return_loss:
        return avg_loss
    else:
        return avg_loss, accuracy, micro_roc_auc, macro_roc_auc, precision, recall, all_labels, all_probs


def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    batch_size = 128
    learning_rate = 0.0005
    num_epochs = 20

    augmentations = {
        'No Augmentation': [],
        'Horizontal Flip': [transforms.RandomHorizontalFlip(p=0.5)],
        'Random Rotation': [transforms.RandomRotation(15)],
        'Random Crop and Flip': [transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip()],
        'Color Jitter': [transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)],
    }

    results = {}  # Store metrics for each augmentation

    for aug_name, aug_transforms in augmentations.items():
        print(f"\nUsing augmentation: {aug_name}")

        data_module = SVHNDataModule(augmentation=aug_transforms)
        train_loader, val_loader, test_loader = data_module.load_data(batch_size)

        model = SmallVGG().to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        train_losses, val_losses = train_model(model, train_loader, val_loader, criterion, optimizer,
                                               num_epochs=num_epochs, device=device)

        test_loss, accuracy, micro_roc_auc, macro_roc_auc, precision, recall, all_labels, all_probs = evaluate_model(model, test_loader, device)

        results[aug_name] = {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
        }

        plt.figure(figsize=(10, 5))
        plt.plot(range(1, len(train_losses) + 1), train_losses, label='Train Loss')
        plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title(f'Train and Validation Loss with {aug_name}')
        plt.legend()
        plt.savefig(f"ex3/train_val_loss_{aug_name.replace(' ', '_')}.png")
        plt.close()

        fpr, tpr, _ = roc_curve(np.eye(10)[all_labels].ravel(), np.array(all_probs).ravel())
        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {micro_roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'ROC Curve with {aug_name}')
        plt.legend(loc="lower right")
        plt.savefig(f"ex3/roc_curve_{aug_name.replace(' ', '_')}.png")
        plt.close()

    # Plot and save the comparison bar chart
    metrics = ['accuracy', 'precision', 'recall']
    plt.figure(figsize=(12, 8))
    for i, metric in enumerate(metrics):
        values = [results[aug][metric] for aug in augmentations.keys()]
        plt.bar(np.arange(len(values)) + i*0.25, values, width=0.25, label=metric)

    plt.xticks(np.arange(len(augmentations)) + 0.25, augmentations.keys(), rotation=45)
    plt.ylabel('Scores')
    plt.title('Comparison of Precision, Recall, and Accuracy using Different Augmentations')
    plt.legend()
    plt.tight_layout()
    plt.savefig("ex3/comparison_bar_chart.png")
    plt.close()


if __name__ == "__main__":
    main()



Using augmentation: No Augmentation
Accuracy: 74.56%, Micro ROC AUC: 0.9602, Macro ROC AUC: 0.9569, Precision: 0.7503, Recall: 0.7456
Epoch [1/20], Train Loss: 1.629542, Val Loss: 0.830281
Accuracy: 81.87%, Micro ROC AUC: 0.9798, Macro ROC AUC: 0.9797, Precision: 0.8291, Recall: 0.8187
Epoch [2/20], Train Loss: 0.632709, Val Loss: 0.586146
Accuracy: 86.36%, Micro ROC AUC: 0.9868, Macro ROC AUC: 0.9862, Precision: 0.8650, Recall: 0.8636
Epoch [3/20], Train Loss: 0.467216, Val Loss: 0.456687
Accuracy: 87.22%, Micro ROC AUC: 0.9889, Macro ROC AUC: 0.9889, Precision: 0.8764, Recall: 0.8722
Epoch [4/20], Train Loss: 0.390242, Val Loss: 0.420422
Accuracy: 89.16%, Micro ROC AUC: 0.9911, Macro ROC AUC: 0.9907, Precision: 0.8921, Recall: 0.8916
Epoch [5/20], Train Loss: 0.344308, Val Loss: 0.367772
Accuracy: 89.72%, Micro ROC AUC: 0.9917, Macro ROC AUC: 0.9914, Precision: 0.8989, Recall: 0.8972
Epoch [6/20], Train Loss: 0.305096, Val Loss: 0.351617
Accuracy: 90.51%, Micro ROC AUC: 0.9929, Macr

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.metrics import precision_score, recall_score
# 创建目录，如果不存在则自动创建
os.makedirs("ex4", exist_ok=True)

# SVHN 数据模块类
class SVHNDataModule:
    def __init__(self):
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.4377, 0.4438, 0.4728), std=(0.1980, 0.2010, 0.1970))
        ])

    def load_data(self, batch_size):
        train_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=self.transform)
        test_dataset = datasets.SVHN(root='./data', split='test', download=True, transform=self.transform)

        train_size = int(0.8 * len(train_dataset))
        val_size = len(train_dataset) - train_size
        train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        return train_loader, val_loader, test_loader

# 定义小型 VGG 模型
class SmallVGG(nn.Module):
    def __init__(self):
        super(SmallVGG, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(8, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(32 * 4 * 4, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x


# 修改训练模型函数，移除早停法的相关代码
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50, device='cuda'):
    model.train()
    device = torch.device(device)
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)

        # 评估模型在验证集上的损失
        val_loss = evaluate_model(model, val_loader, device, return_loss=True)
        val_losses.append(val_loss)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

    return train_losses, val_losses
def evaluate_model(model, data_loader, device='cuda', return_loss=False):
    model.eval()
    running_loss = 0.0
    all_labels = []
    all_preds = []
    all_probs = []
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            probs = torch.softmax(outputs, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = running_loss / len(data_loader)
    accuracy = np.mean(np.array(all_labels) == np.array(all_preds))

    num_classes = 10
    all_labels_onehot = np.eye(num_classes)[all_labels]
    all_probs = np.array(all_probs)

    micro_roc_auc = roc_auc_score(all_labels_onehot, all_probs, average='micro')
    macro_roc_auc = roc_auc_score(all_labels_onehot, all_probs, average='macro')

    print(f'Accuracy: {accuracy * 100:.2f}%, Micro ROC AUC: {micro_roc_auc:.4f}, Macro ROC AUC: {macro_roc_auc:.4f}')

    if return_loss:
        return avg_loss
    else:
        return avg_loss, accuracy, micro_roc_auc, macro_roc_auc, all_labels, all_preds, all_probs

# 主函数，增加保存 Precision、Recall 和 Accuracy 柱状图的代码
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    batch_size = 128
    learning_rate = 0.0005
    num_epochs = 30

    # 数据加载
    data_module = SVHNDataModule()
    train_loader, val_loader, test_loader = data_module.load_data(batch_size)

    # 定义优化器列表
    optimizers = {
        'SGD': optim.SGD,
        'Adam': optim.Adam,
        'RMSprop': optim.RMSprop,
        'AdamW': optim.AdamW
    }

    # 存储每个优化器的评估指标
    results = {}

    # 遍历不同的优化器
    for opt_name, opt_class in optimizers.items():
        print(f"\nUsing optimizer: {opt_name}")

        # 初始化模型、损失函数和优化器
        model = SmallVGG().to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = opt_class(model.parameters(), lr=learning_rate)

        # 训练模型
        train_losses, val_losses = train_model(model, train_loader, val_loader, criterion, optimizer,
                                               num_epochs=num_epochs, device=device)

        # 评估模型在测试集上的表现
        test_loss, accuracy, micro_roc_auc, macro_roc_auc, all_labels, all_preds, all_probs = evaluate_model(model, test_loader, device)

        # 计算 Precision 和 Recall
        all_labels = np.array(all_labels)
        all_preds = np.array(all_preds)

        precision = precision_score(all_labels, all_preds, average='weighted', labels=np.arange(10))
        recall = recall_score(all_labels, all_preds, average='weighted', labels=np.arange(10))

        # 保存当前优化器的指标
        results[opt_name] = {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall
        }

        # 绘制并保存 Train Loss 和 Validation Loss
        plt.figure(figsize=(10, 5))
        plt.plot(range(1, len(train_losses) + 1), train_losses, label='Train Loss')
        plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title(f'Train and Validation Loss with {opt_name}')
        plt.legend()
        plt.savefig(f"ex4/train_val_loss_{opt_name}.png")
        plt.close()

        # 绘制并保存 ROC 曲线
        fpr, tpr, _ = roc_curve(np.eye(10)[all_labels].ravel(), np.array(all_probs).ravel())
        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {micro_roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'ROC Curve with {opt_name}')
        plt.legend(loc="lower right")
        plt.savefig(f"ex4/roc_curve_{opt_name}.png")
        plt.close()

        # 绘制并保存混淆矩阵
        cm = confusion_matrix(all_labels, all_preds)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title(f'Confusion Matrix with {opt_name}')
        plt.savefig(f"ex4/confusion_matrix_{opt_name}.png")
        plt.close()

    # 绘制并保存 Precision、Recall 和 Accuracy 的柱状对比图
    metrics = ['accuracy', 'precision', 'recall']
    plt.figure(figsize=(12, 8))
    for i, metric in enumerate(metrics):
        values = [results[opt][metric] for opt in optimizers.keys()]
        plt.bar(np.arange(len(values)) + i*0.25, values, width=0.25, label=metric)

    plt.xticks(np.arange(len(optimizers)) + 0.25, optimizers.keys(), rotation=45)
    plt.ylabel('Scores')
    plt.title('Comparison of Precision, Recall, and Accuracy using Different Optimizers')
    plt.legend()
    plt.tight_layout()
    plt.savefig("ex4/comparison_bar_chart.png")
    plt.close()

if __name__ == "__main__":
    main()



Using optimizer: SGD
Accuracy: 9.45%, Micro ROC AUC: 0.5236, Macro ROC AUC: 0.4955
Epoch [1/30], Train Loss: 2.301729, Val Loss: 2.299449
Accuracy: 9.45%, Micro ROC AUC: 0.5855, Macro ROC AUC: 0.4955
Epoch [2/30], Train Loss: 2.297291, Val Loss: 2.295136
Accuracy: 9.45%, Micro ROC AUC: 0.5877, Macro ROC AUC: 0.4976
Epoch [3/30], Train Loss: 2.293133, Val Loss: 2.291092
Accuracy: 18.65%, Micro ROC AUC: 0.5978, Macro ROC AUC: 0.4972
Epoch [4/30], Train Loss: 2.289220, Val Loss: 2.287282
Accuracy: 18.65%, Micro ROC AUC: 0.6036, Macro ROC AUC: 0.4967
Epoch [5/30], Train Loss: 2.285550, Val Loss: 2.283706
Accuracy: 18.65%, Micro ROC AUC: 0.6034, Macro ROC AUC: 0.4957
Epoch [6/30], Train Loss: 2.282106, Val Loss: 2.280359
Accuracy: 18.65%, Micro ROC AUC: 0.6034, Macro ROC AUC: 0.4958
Epoch [7/30], Train Loss: 2.278872, Val Loss: 2.277198
Accuracy: 18.65%, Micro ROC AUC: 0.6052, Macro ROC AUC: 0.4958
Epoch [8/30], Train Loss: 2.275810, Val Loss: 2.274223
Accuracy: 18.65%, Micro ROC AUC: 0.60

E:\envs\cisc3024_venv1\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Using optimizer: Adam
Accuracy: 66.95%, Micro ROC AUC: 0.9417, Macro ROC AUC: 0.9385
Epoch [1/30], Train Loss: 1.811828, Val Loss: 1.009995
Accuracy: 77.39%, Micro ROC AUC: 0.9704, Macro ROC AUC: 0.9701
Epoch [2/30], Train Loss: 0.802329, Val Loss: 0.707372
Accuracy: 83.81%, Micro ROC AUC: 0.9838, Macro ROC AUC: 0.9826
Epoch [3/30], Train Loss: 0.575788, Val Loss: 0.514666
Accuracy: 86.75%, Micro ROC AUC: 0.9882, Macro ROC AUC: 0.9873
Epoch [4/30], Train Loss: 0.449839, Val Loss: 0.434817
Accuracy: 87.37%, Micro ROC AUC: 0.9891, Macro ROC AUC: 0.9891
Epoch [5/30], Train Loss: 0.383677, Val Loss: 0.412607
Accuracy: 88.41%, Micro ROC AUC: 0.9906, Macro ROC AUC: 0.9903
Epoch [6/30], Train Loss: 0.341747, Val Loss: 0.380017
Accuracy: 89.75%, Micro ROC AUC: 0.9919, Macro ROC AUC: 0.9914
Epoch [7/30], Train Loss: 0.310378, Val Loss: 0.347556
Accuracy: 90.04%, Micro ROC AUC: 0.9923, Macro ROC AUC: 0.9919
Epoch [8/30], Train Loss: 0.285270, Val Loss: 0.343686
Accuracy: 90.18%, Micro ROC AUC: 

In [5]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score, precision_score, recall_score, confusion_matrix, roc_curve
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


os.makedirs("ex5", exist_ok=True)


class SVHNDataModule:
    def __init__(self):
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.4377, 0.4438, 0.4728), std=(0.1980, 0.2010, 0.1970))
        ])

    def load_data(self):
        train_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=self.transform)
        test_dataset = datasets.SVHN(root='./data', split='test', download=True, transform=self.transform)
        train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)
        return train_loader, test_loader

# 定义小型 VGG 模型，支持不同的 Dropout 率
class SmallVGG(nn.Module):
    def __init__(self, dropout_rate_conv=0.25, dropout_rate_fc=0.5):
        super(SmallVGG, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(8, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(dropout_rate_conv),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(dropout_rate_conv),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(dropout_rate_conv)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(32 * 4 * 4, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate_fc),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

# 训练模型的函数，返回训练中的所有损失值
def train_model(model, train_loader, criterion, optimizer, num_epochs=10, device='cuda'):
    model.train()
    device = torch.device(device)
    train_losses = []

    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        train_losses.append(epoch_loss)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.6f}')

    return train_losses

# 评估模型的函数
def evaluate_model_metrics(model, test_loader, criterion, device='cuda'):
    model.eval()
    all_labels = []
    all_preds = []
    all_probs = []
    test_loss = 0.0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            probs = torch.softmax(outputs, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    test_loss /= len(test_loader)
    accuracy = np.mean(np.array(all_labels) == np.array(all_preds))
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    cm = confusion_matrix(all_labels, all_preds)

    return test_loss, accuracy, precision, recall, cm, all_labels, all_probs

# 生成 ROC 曲线的函数
def plot_roc_curve(all_labels, all_probs, dropout):
    plt.figure()
    for i in range(10):
        fpr, tpr, _ = roc_curve(np.array(all_labels) == i, np.array(all_probs)[:, i])
        auc_score = roc_auc_score(np.array(all_labels) == i, np.array(all_probs)[:, i])
        plt.plot(fpr, tpr, label=f'Class {i} (AUC = {auc_score:.2f})')

    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve (Dropout {dropout})')
    plt.legend()
    plt.savefig(f'ex5/roc_curve_dropout_{dropout}.png')
    plt.close()

# 主函数，测试不同 Dropout 率并保存图表
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data_module = SVHNDataModule()
    train_loader, test_loader = data_module.load_data()
    dropout_rates = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]

    results = {}

    for dropout in dropout_rates:
        print(f"\nUsing Dropout Rate - Conv & FC: {dropout}")
        model = SmallVGG(dropout_rate_conv=dropout, dropout_rate_fc=dropout).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.0005)

        # 训练模型并记录损失
        train_losses = train_model(model, train_loader, criterion, optimizer, num_epochs=50, device=device)

        # 评估模型并获得所有预测的概率
        test_loss, accuracy, precision, recall, cm, all_labels, all_probs = evaluate_model_metrics(model, test_loader, criterion, device)

        results[dropout] = {
            "train_loss": train_losses[-1],
            "test_loss": test_loss,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "cm": cm
        }

        # 绘制并保存训练过程的 Loss 曲线
        plt.figure()
        plt.plot(range(1, len(train_losses) + 1), train_losses, label="Train Loss")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.title(f"Loss Curve (Dropout {dropout})")
        plt.legend()
        plt.savefig(f"ex5/loss_curve_dropout_{dropout}.png")
        plt.close()

        # 绘制并保存 ROC 曲线
        plot_roc_curve(all_labels, all_probs, dropout)

        # 保存混淆矩阵
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, xticklabels=range(10), yticklabels=range(10))
        plt.xlabel("Predicted Label")
        plt.ylabel("True Label")
        plt.title(f"Confusion Matrix (Dropout {dropout})")
        plt.savefig(f"ex5/confusion_matrix_dropout_{dropout}.png")
        plt.close()

    # 绘制并保存不同 Dropout 率的指标柱状图
    metrics = ['train_loss', 'test_loss', 'accuracy', 'precision', 'recall']
    plt.figure(figsize=(15, 8))
    for i, metric in enumerate(metrics):
        values = [results[dropout][metric] for dropout in dropout_rates]
        plt.bar(np.arange(len(dropout_rates)) + i * 0.15, values, width=0.15, label=metric)

    plt.xticks(np.arange(len(dropout_rates)) + 0.3, [f"Dropout {d}" for d in dropout_rates], rotation=45)
    plt.ylabel('Scores')
    plt.title('Comparison of Metrics with Different Dropout Rates')
    plt.legend()
    plt.tight_layout()
    plt.savefig("ex5/metrics_comparison_bar_chart.png")
    plt.close()

if __name__ == "__main__":
    main()



Using Dropout Rate - Conv & FC: 0.1
Epoch [1/50], Loss: 1.562087
Epoch [2/50], Loss: 0.637070
Epoch [3/50], Loss: 0.473867
Epoch [4/50], Loss: 0.402071
Epoch [5/50], Loss: 0.361342
Epoch [6/50], Loss: 0.326549
Epoch [7/50], Loss: 0.304580
Epoch [8/50], Loss: 0.282998
Epoch [9/50], Loss: 0.267809
Epoch [10/50], Loss: 0.253569
Epoch [11/50], Loss: 0.242066
Epoch [12/50], Loss: 0.229694
Epoch [13/50], Loss: 0.218907
Epoch [14/50], Loss: 0.210744
Epoch [15/50], Loss: 0.200336
Epoch [16/50], Loss: 0.191889
Epoch [17/50], Loss: 0.182928
Epoch [18/50], Loss: 0.177726
Epoch [19/50], Loss: 0.171618
Epoch [20/50], Loss: 0.161420
Epoch [21/50], Loss: 0.158741
Epoch [22/50], Loss: 0.151599
Epoch [23/50], Loss: 0.146770
Epoch [24/50], Loss: 0.141287
Epoch [25/50], Loss: 0.140585
Epoch [26/50], Loss: 0.136619
Epoch [27/50], Loss: 0.129486
Epoch [28/50], Loss: 0.126266
Epoch [29/50], Loss: 0.124861
Epoch [30/50], Loss: 0.121121
Epoch [31/50], Loss: 0.115890
Epoch [32/50], Loss: 0.113719
Epoch [33/50